### Codio Activity 9.7: Ridge vs. Sequential Feature Selection

**Expected Time: 60 Minutes**

**Total Points: 40**

This activity focuses on comparing the results of a `Ridge` regression model with that of a `LinearRegression` model built using `SequentialFeatureSelector`.  Both of these approaches seek to limit the complexity of the model.  The `Ridge` estimator applies a penalty that shrinks the coefficients of the model while using the `SequentialFeatureSelector` selects a subset of features to build a model with.  

#### Index

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)


In [1]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display="diagram")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### The Insurance Data

For this example, we return to the insurance data with cubic features.  Below the train and test data is loaded and the train and test sets are determined.  Recall that the target feature has the logarithm applied to it.  

In [2]:
train = pd.read_csv('data/train_cubic.csv')
test = pd.read_csv('data/test_cubic.csv')

In [3]:
X_train, y_train = train.drop('target_log', axis = 1), train['target_log']
X_test, y_test = test.drop('target_log', axis = 1), test['target_log']

### Problem 1

#### Feature Selection Pipeline

**10 Points**

To begin, use the pipeline below to construct a grid search over the `n_features_to_select` parameter of the `SequentialFeatureSelector` transformer.  Consider 2, 3, 4, and 5 features in your search.  Create the dictionary to be used in the search as `param_dict`.  

Assign your grid to `selector_grid`, fit on the training data, and determine the mean squared error on the train and test set.  Assign the errors as floats to `selector_train_mse` and `selector_test_mse` respectively.

In [13]:
selector_pipe = Pipeline([('selector', SequentialFeatureSelector(LinearRegression())), ('model', LinearRegression())])
selector_pipe

Pipeline(steps=[('selector',
                 SequentialFeatureSelector(estimator=LinearRegression())),
                ('model', LinearRegression())])

In [5]:
### GRADED

param_dict = {}
selector_grid = ''
selector_train_mse = ''
selector_test_mse = ''

### BEGIN SOLUTION
param_dict = {'selector__n_features_to_select': [2, 3, 4, 5]}
selector_grid = GridSearchCV(selector_pipe, param_grid=param_dict)
selector_grid.fit(X_train, y_train)
train_preds = selector_grid.predict(X_train)
test_preds = selector_grid.predict(X_test)
selector_train_mse = mean_squared_error(y_train, train_preds)
selector_test_mse = mean_squared_error(y_test, test_preds)
### END SOLUTION

# ANSWER CHECK
print(f'Train MSE: {selector_train_mse}')
print(f'Test MSE: {selector_test_mse}')

Train MSE: 0.6031734290034885
Test MSE: 0.5655875591380699


In [6]:
### BEGIN HIDDEN TESTS
selector_pipe_ = Pipeline([('selector', SequentialFeatureSelector(LinearRegression())),
                         ('model', LinearRegression())])
param_dict_ = {'selector__n_features_to_select': [2, 3, 4, 5]}
selector_grid_ = GridSearchCV(selector_pipe_, param_grid=param_dict_)
selector_grid_.fit(X_train, y_train)
train_preds_ = selector_grid_.predict(X_train)
test_preds_ = selector_grid_.predict(X_test)
selector_train_mse_ = mean_squared_error(y_train, train_preds_)
selector_test_mse_ = mean_squared_error(y_test, test_preds_)
#
#
#
assert list(param_dict_.values()) == list(param_dict.values()), 'Check the parameter dictionary -- double underscores after selector'
assert selector_train_mse == selector_train_mse_
assert selector_test_mse == selector_test_mse_
### END HIDDEN TESTS

### Problem 2

#### Ridge Grid

**10 Points**

Now, construct a `Pipeline` that contains two steps -- `scaler` and `ridge` that first standard scales the data and then build a ridge regression model.  Assign your pipeline as `ridge_pipe`.  Use this to execute the grid search over the `alpha` hyperparameter of the `Ridge` estimator using the training data. Determine the mean squared error on the train and test data. 

Assign the errors as floats to `ridge_train_mse` and `ridge_test_mse` respectively.

In [14]:
### GRADED

ridge_param_dict = ''
ridge_pipe = ''
ridge_grid = ''
ridge_train_mse = ''
ridge_test_mse = ''

### BEGIN SOLUTION
ridge_param_dict = {'ridge__alpha': np.logspace(0, 10, 50)}
ridge_pipe = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge())])
ridge_grid = GridSearchCV(ridge_pipe, param_grid=ridge_param_dict)
ridge_grid.fit(X_train, y_train)
ridge_train_preds = ridge_grid.predict(X_train)
ridge_test_preds = ridge_grid.predict(X_test)
ridge_train_mse = mean_squared_error(y_train, ridge_train_preds)
ridge_test_mse = mean_squared_error(y_test, ridge_test_preds)
### END SOLUTION

# ANSWER CHECK
print(f'Train MSE: {ridge_train_mse}')
print(f'Test MSE: {ridge_test_mse}')
ridge_pipe

Train MSE: 0.5870277750390874
Test MSE: 0.5532169282339904


Pipeline(steps=[('scaler', StandardScaler()), ('ridge', Ridge())])

In [8]:
### BEGIN HIDDEN TESTS
ridge_param_dict_ = {'ridge__alpha': np.logspace(0, 10, 50)}
ridge_pipe_ = Pipeline([('scaler', StandardScaler()), 
                      ('ridge', Ridge())])
ridge_grid_ = GridSearchCV(ridge_pipe_, param_grid=ridge_param_dict_)
ridge_grid_.fit(X_train, y_train)
ridge_train_preds_ = ridge_grid_.predict(X_train)
ridge_test_preds_ = ridge_grid_.predict(X_test)
ridge_train_mse_ = mean_squared_error(y_train, ridge_train_preds_)
ridge_test_mse_ = mean_squared_error(y_test, ridge_test_preds_)
#
#
#
np.testing.assert_array_equal(ridge_param_dict_['ridge__alpha'], ridge_param_dict['ridge__alpha'])
assert ridge_train_mse == ridge_train_mse_
assert ridge_test_mse == ridge_test_mse_
### END HIDDEN TESTS

### Problem 3

#### Examining the "best" model

**10 Points**

Your results should suggest that the model using the sequential feature selector and `LinearRegression` estimator.  This was fit with the object `selector_grid`.  One question we may have is what was the optimal number of features selected and what were they?  

Use the `selector_grid` to extract both the feature names and their associated coefficients.  This will involve:

- `.best_estimator_`: extract the best estimator/selector pair from your grid search
- `.named_steps['selector']`: extract the selector from the pipeline
- `.named_steps['model']`: extract the model from the pipeline
- `.get_support()`: extract best features from selector.  This returns booleans as to whether feature was selected, we can use this to slice our train data.  

```python
X_train.columns[best_selector.get_support()]
```

- `.coef_`: coefficients from best model

In [9]:
### GRADED

best_estimator = ''
best_selector = ''
best_model = ''
feature_names = ''
coefs = ''


### BEGIN SOLUTION
best_estimator = selector_grid.best_estimator_
best_selector = best_estimator.named_steps['selector']
best_model = selector_grid.best_estimator_.named_steps['model']
feature_names = X_train.columns[best_selector.get_support()]
coefs = best_model.coef_
### END SOLUTION

# Answer check
print(best_estimator)
print(f'Features from best selector: {feature_names}.')
print('Coefficient values: ')
print('===================')
pd.DataFrame([coefs.T], columns = feature_names, index = ['model'])

Pipeline(steps=[('selector',
                 SequentialFeatureSelector(estimator=LinearRegression(),
                                           n_features_to_select=2)),
                ('model', LinearRegression())])
Features from best selector: Index(['age', 'bmi children'], dtype='object').
Coefficient values: 


,age,bmi children
model,0.032852,0.00368


In [10]:
### BEGIN HIDDEN TESTS
best_estimator_ = selector_grid_.best_estimator_
best_selector_ = best_estimator_.named_steps['selector']
best_model_ = selector_grid_.best_estimator_.named_steps['model']
feature_names_ = X_train.columns[best_selector_.get_support()]
coefs_ = best_model_.coef_
#
#
#
np.testing.assert_array_equal(coefs, coefs_)
np.testing.assert_array_equal(feature_names, feature_names_)
### END HIDDEN TESTS

### Problem 4

#### Comparing observations 

**10 Points**

According to your model, predict the billed costs for person 1 and person 2 below:

- **Person 1**: Age = 30, bmi = 40, children = 0
- **Person 2**: Age = 45, bmi = 50, children = 2

Use the information from **Problem 3** and the model coefficients to make these predictions.

Note that you will want to transform your predictions.  From your model the predictions are in terms of the logarithm of cost.  To transform the logarithm to the actual value, use `np.exp` -- the inverse of a logarithm. Assign your predictions as floats to `person1` and `person2` below.  Your solution will be checked to two decimal point accuracy. 

In [11]:
### GRADED
ages = [30, 45]
bmis = [40, 50]
childrens = [0, 2]
person1 = ''
person2 = ''

### BEGIN SOLUTION
person1 = np.exp(best_model.intercept_ + coefs[0]*ages[0] + coefs[1]*(bmis[0]*childrens[0]))
person2 = np.exp(best_model.intercept_ + coefs[0]*ages[1] + coefs[1]*(bmis[1]*childrens[1]))
### END SOLUTION

# Answer check
print(f'The difference between Person 1 and Person 2 is {person2 - person1: .2f}')

The difference between Person 1 and Person 2 is  8052.04


In [12]:
### BEGIN HIDDEN TESTS
ages = [30, 45]
bmis = [40, 50]
childrens = [0, 2]
person1_ = np.exp(best_model.intercept_ + coefs[0]*ages[0] + coefs[1]*(bmis[0]*childrens[0]))
person2_ = np.exp(best_model.intercept_ + coefs[0]*ages[1] + coefs[1]*(bmis[1]*childrens[1]))
#
#
#
assert round(float(person1), 2) == round(float(person1_), 2), 'Person 1 is not correct.'
assert round(float(person2), 2) == round(float(person2_), 2), 'Person 2 is not correct.'
### END HIDDEN TESTS

The models here could be revisited and more encoding of features and different polynomial terms can be incorporated.  More important is understanding how to construct the pipelines and interrogate the resulting models to understand what they say about your data.  Does having a higher body mass matter if one does not have children?  Does this seem reasonable?